<a href="https://colab.research.google.com/github/bhkwon89/POZALABS_test/blob/main/musicVAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AE vs VAE  

AutoEncoder는 데이터의 차원축소(feature 추출)를 위해 사용한다. (ex color 영상에서 depth를 뽑아내기)
encoder의 출력이 feature이며 특정 값으로 떨어진다.
예를 들어 개, 사람, 자동차, 비행기가 있는 데이터 셋을 넣고 latent vector(=feature)를 1차원으로 만들면 개,사람 - 자동차,비행기로 나뉘어 질 것이다. 해당 모델에 만약 코끼리 사진을 입력한 경우 어떻게 될까? latent vector는 생물이므로 개,사람 쪽에 분류가 될 것이지만 decoding한 출력 값은 엉뚱한 결과 값이 나올 것이다. 오버피팅이 일어나는 것이다.


반면 Variational AutoEncoder의 목적은 데이터를 생성함에 있다. 학습하지 않았던 데이터를 입력해도 학습시킨 데이터 같은 출력이 비스무리하게 나오면 된다.
AE에서 발생하는 오버피팅을 피해야 하기 때문에 latent vector 값을 특정 값이 아닌 평균과 표준편차로 변형시키되 각각의 평균들의 분포가 너무 멀리 떨어져 있을 경우 사실 상 값과 다를 바가 없다. (면을 멀리서 보면 점이듯) 하여 평균들의 분포가 표준편차를 이루도록 학습한다.




# MusicVAE
위에서 설명한 VAE를 음악에 접목시킨 것으로 encoder로는 bidirectional-LSTM, decoder로는 GrooveLSTM이라는 계층적 디코더를 사용했다.

논문에서는 decoder로 RNN만 사용할 경우 성능이 않좋았은데 출력 sequence를 만들어지면 latent vector의 영향력이 줄어들어서 그럴 것이라고 예상했다.
LSTM으로 conductor를 생성하고 다시 LSTM으로 복원한다.  
이 형태를 1마디의 대표 note를 생성(conductor)하고 1마디의 대표 note에서 16노트로 생성하는 식이라 생각된다.

# magenta 패키지 설치
패키지 내 mudule간 의존성 문제로 인해 아래와 같은 방식으로 해결  
출처 https://test.ocom.vn/?url=github.com/magenta/magenta/issues/2045

In [1]:
!pip install hmmlearn

!apt-get update -y
!apt-get install python3.8
!update-alternatives --set python3 /usr/bin/python3.8
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python get-pip.py

import sys

_ = (sys.path.append("/usr/local/lib/python3.8/dist-packages"))

!pip install numba==0.48
!pip install numpy==1.23
!pip install packaging>=21.3
!pip install librosa==0.7.2

!pip install magenta==2.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 4.1 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [77.6 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRele

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.4/268.4 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 7

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/groove/info.csv')
df = pd.DataFrame(df)
df

FileNotFoundError: ignored

In [ ]:
convert_directory(data_root,tfrec_root,recursive=True) # midi to tfrecord

아래 소스코드는 https://github.com/oobinkim/MusicVAE_Groove 링크를 참조하였습니다.

In [ ]:
#configs.py 스크립트 일부 발췌

class Config(collections.namedtuple(
    'Config',
    ['model', 'hparams', 'note_sequence_augmenter', 'data_converter',
     'train_examples_path', 'eval_examples_path', 'tfds_name'])):

    def values(self):
        return self._asdict()

Config.__new__.__defaults__ = (None,) * len(Config._fields)


def update_config(config, update_dict):
    config_dict = config.values()
    config_dict.update(update_dict)
    return Config(**config_dict)


CONFIG_MAP = {}


HParams = contrib_training.HParams

#=== 모델구조 ===
CONFIG_MAP['groovae_4bar'] = Config(
    model=MusicVAE(lstm_models.BidirectionalLstmEncoder(), #BidirectionalLstmEncoder 사용 
                   lstm_models.GrooveLstmDecoder()), #Hierarchical Decoder
    hparams=merge_hparams(
        lstm_models.get_default_hparams(),
        HParams(
            batch_size=512, #데이터 배치사이즈
            max_seq_len=16 * 4,  # 4마디 길이지정
            z_size=256, #잠재백터 사이즈
            enc_rnn_size=[512], #인코더 순환 사이즈지정
            dec_rnn_size=[256, 256],#디코더 순환사이즈 지정
            max_beta=0.2,
            free_bits=48,
            dropout_keep_prob=0.3,#드롭아웃
        )),
    note_sequence_augmenter=None,
    data_converter=data.GrooveConverter(
        split_bars=4, steps_per_quarter=4, quarters_per_bar=4,
        max_tensors_per_notesequence=20,
        pitch_classes=data.ROLAND_DRUM_PITCH_CLASSES,

        inference_pitch_classes=data.REDUCED_DRUM_PITCH_CLASSES),
    # tfds_name='groove/4bar-midionly',
    train_examples_path='/content/drive/MyDrive/groove/music.tfrecord', #데이터 경로 설정
)

## 훈련정의 소스코드 

In [ ]:
#============ License=====================================================
# Copyright 2022 The Magenta Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#=======================================================================

#=== MusicVAE train script===
#=== magenta/models/music_vae/music_vae_train.py ===

# Should not be called from within the graph to avoid redundant summaries.
def _trial_summary(hparams, examples_path, output_dir):
  #=== 텐서보드 summary 텍스트===

    examples_path_summary = tf.summary.text(
      'examples_path', tf.constant(examples_path, name='examples_path'),
      collections=[])
    hparams_dict = hparams.values()

#=== 하이퍼 파라미터===
  # Create a markdown table from hparams.
    header = '| Key | Value |\n| :--- | :--- |\n'
    keys = sorted(hparams_dict.keys())
    lines = ['| %s | %s |' % (key, str(hparams_dict[key])) for key in keys]
    hparams_table = header + '\n'.join(lines) + '\n'

    hparam_summary = tf.summary.text(
      'hparams', tf.constant(hparams_table, name='hparams'), collections=[])

    with tf.Session() as sess:
        writer = tf.summary.FileWriter(output_dir, graph=sess.graph)
        writer.add_summary(examples_path_summary.eval())
        writer.add_summary(hparam_summary.eval())
        writer.close()


def _get_input_tensors(dataset, config):
  #== 데이터로부터 텐서 입력 ===
    batch_size = config.hparams.batch_size
    iterator = tf.data.make_one_shot_iterator(dataset)
    (input_sequence, output_sequence, control_sequence,
    sequence_length) = iterator.get_next()
    input_sequence.set_shape(
    [batch_size, None, config.data_converter.input_depth])
    output_sequence.set_shape(
    [batch_size, None, config.data_converter.output_depth])
    
    if not config.data_converter.control_depth:
        control_sequence = None
    
    else:
        control_sequence.set_shape(
            [batch_size, None, config.data_converter.control_depth])
        sequence_length.set_shape([batch_size] + sequence_length.shape[1:].as_list())
        
    return {
        'input_sequence': input_sequence,
        'output_sequence': output_sequence,
        'control_sequence': control_sequence,
        'sequence_length': sequence_length
    }

#=== 훈련 체크포인트, 시간 설정===
def train(train_dir,
          config,
          dataset_fn,
          checkpoints_to_keep=5,
          keep_checkpoint_every_n_hours=1,
          num_steps=None,
          master='',
          num_sync_workers=0,
          num_ps_tasks=0,
          task=0):

#==== train loop ====
    tf.gfile.MakeDirs(train_dir)
    is_chief = (task == 0)

    with tf.Graph().as_default():
        with tf.device(tf.train.replica_device_setter(
            num_ps_tasks, merge_devices=True)):
            
            model = config.model
            model.build(config.hparams,
                        config.data_converter.output_depth,
                        is_training=True)
            #== 옵티마이저 ===
            optimizer = model.train(**_get_input_tensors(dataset_fn(), config))

            hooks = []
            if num_sync_workers:
                optimizer = tf.train.SyncReplicasOptimizer(
                    optimizer,num_sync_workers)
                hooks.append(optimizer.make_session_run_hook(is_chief))

            grads, var_list = list(zip(*optimizer.compute_gradients(model.loss)))
            global_norm = tf.global_norm(grads)
            tf.summary.scalar('global_norm', global_norm)
            
            if config.hparams.clip_mode == 'value':
                g = config.hparams.grad_clip
                clipped_grads = [tf.clip_by_value(grad, -g, g) for grad in grads]
            elif config.hparams.clip_mode == 'global_norm':
                clipped_grads = tf.cond(
                    global_norm < config.hparams.grad_norm_clip_to_zero,
                    lambda: tf.clip_by_global_norm(  # pylint:disable=g-long-lambda
                        grads, config.hparams.grad_clip, use_norm=global_norm)[0],
                    lambda: [tf.zeros(tf.shape(g)) for g in grads])
            else:
                raise ValueError(
                    'Unknown clip_mode: {}'.format(config.hparams.clip_mode))
            train_op = optimizer.apply_gradients(
                    list(zip(clipped_grads, var_list)),
                    global_step=model.global_step,
                    name='train_step')
            logging_dict = {'global_step': model.global_step,
                            'loss': model.loss}
            
            hooks.append(tf.train.LoggingTensorHook(logging_dict, every_n_iter=100))
            if num_steps:
                hooks.append(tf.train.StopAtStepHook(last_step=num_steps))
                
            scaffold = tf.train.Scaffold(
                saver=tf.train.Saver(
                    max_to_keep=checkpoints_to_keep,
                    keep_checkpoint_every_n_hours=keep_checkpoint_every_n_hours))
            
            tf_slim.training.train(
                train_op=train_op,
                logdir=train_dir,
                scaffold=scaffold,
                hooks=hooks,
                save_checkpoint_secs=60,
                master=master,
                is_chief=is_chief)


def run(config_map,
        tf_file_reader=tf.data.TFRecordDataset,
        file_reader=tf.python_io.tf_record_iterator,
        is_training=True):
    config = config_map['groovae_4bar']
    train_dir = '/content/drive/MyDrive/groove/train'
    num_steps = 5000 #훈련 epoch
    
    def dataset_fn():
        return data.get_dataset(
            config,
            tf_file_reader=tf_file_reader,
            is_training=True,
            cache_dataset=True)
    
    if is_training == True:
        train(
            train_dir,
            config=config,
            dataset_fn=dataset_fn,
            num_steps=num_steps)      
    
    else:
        print("EVAL")


In [ ]:
CONFIG_MAP['groovae_4bar'].hparams.clip_mode

## 훈련 RUN

In [ ]:
run(CONFIG_MAP) #epoch 5000

# 생성하기

In [ ]:
tf.compat.v1.enable_eager_execution()

In [ ]:
#=== magenta/magenta/models/music_vae/music_vae_generate.py ===


model = TrainedModel(
    config=CONFIG_MAP['groovae_4bar'],
    batch_size=1,
    checkpoint_dir_or_path='/content/drive/MyDrive/groove/train') # 체크포인트의 경로

generated_sequence = model.sample(n=1, length=16*4, temperature=0.5)
note_seq.sequence_proto_to_midi_file(generated_sequence[0], '/content/drive/MyDrive/groove/gen_midi/drum_4bar.mid')